In [1]:
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import model.loss as module_loss
import model.metric as module_metric
import model.model as module_arch
from parse_config import ConfigParser
import argparse
import pandas as pd

In [2]:
# best model path
bast_model_path = 'saved/models/ConstellationClassification/1120_131506/model_best.pth'
# 读入配置文件
parser = argparse.ArgumentParser(description='PyTorch Template')
parser.add_argument('-c', '--config', default=None, type=str,
                  help='config file path (default: None)')
parser.add_argument('-r', '--resume', default=bast_model_path,
                    type=str, help='path to latest checkpoint (default: None)')
parser.add_argument('-d', '--device', default=None, type=str,
                    help='indices of GPUs to enable (default: all)')
args = parser.parse_args(args=['--resume', bast_model_path, '--device', 'all'])
config = ConfigParser.from_args(args, jupyter=True)

# 现有的类别对应编号
label_map = {'4--PSK':0, '8--QAM':1, '16-QAM':2, '32-QAM':3, '64-QAM':4}
label_inv_map = {v: k for k, v in label_map.items()}


In [3]:
# 设置logger
logger = config.get_logger('test')
# 获取dataloader
data_loader = getattr(module_data, config['data_loader']['type'])(
    batch_size=1,
    shuffle=False,
    validation_split=0.0,
    training=False,
    num_workers=2
)
# 设置model
model = config.init_obj('arch', module_arch)
# logger.info(model)
# 设置损失函数等
loss_fn = getattr(module_loss, config['loss'])
metric_fns = [getattr(module_metric, met) for met in config['metrics']]
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '-1'
logger.info('Loading checkpoint: {} ...'.format(config.resume))
checkpoint = torch.load(config.resume)
state_dict = checkpoint['state_dict']
if config['n_gpu'] > 1:
    model = torch.nn.DataParallel(model)
model.load_state_dict(state_dict)
# prepare model for testing
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

total_loss = 0.0
total_metrics = torch.zeros(len(metric_fns))

/home/ubuntu/miniconda3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/miniconda3/envs/d2l/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading checkpoint: saved/models/ConstellationClassification/1120_131506/model_best.pth ...


In [4]:
results_list = []
with torch.no_grad():
    for i, (data, target, noisy_value) in enumerate(tqdm(data_loader)):
        result = []
        data, target = data.to(device), target.to(device)
        output = model(data)
        #
        # save sample images, or do something with output here
        #
        # computing loss, metrics on test set
        loss = loss_fn(output, target)
        batch_size = data.shape[0]
        total_loss += loss.item() * batch_size
        result.append(target[0].item())
        result.append(noisy_value[0])
        result.append(loss.item() * batch_size)
        for i, metric in enumerate(metric_fns):
            metric_result = metric(output, target)
            total_metrics[i] += metric_result * batch_size
            result.append(metric_result)
        results_list.append(result)

# 转化为dataframe
results_df = pd.DataFrame(results_list, columns=['label', 'noisy_value', 'loss', 'acc', 'top-k acc'])
results_df['label'] = results_df['label'].map(label_inv_map)
            
        

100%|██████████| 600/600 [00:04<00:00, 123.34it/s]


In [5]:
n_samples = len(data_loader.sampler)
log = {'loss': total_loss / n_samples}
log.update({
    met.__name__: total_metrics[i].item() / n_samples for i, met in enumerate(metric_fns)
})
logger.info(log)

{'loss': -1266.937238260905, 'accuracy': 0.955, 'top_k_acc': 1.0}


In [6]:
results_df.groupby('label').mean()

,noisy_value,loss,acc,top-k acc
label,,,,
16-QAM,16.0,-1417.360253,0.983333,1.0
32-QAM,16.0,-1440.760075,0.991667,1.0
4--PSK,16.0,-1216.536678,1.000000,1.0
64-QAM,16.0,-1090.745926,0.808333,1.0
8--QAM,16.0,-1169.283259,0.991667,1.0


In [13]:
results_df.groupby('noisy_value').mean()

,loss,acc,top-k acc
noisy_value,,,
2.0,-1129.91614136,0.80,1.0
5.0,-1181.81492798,0.82,1.0
7.0,-1219.88642700,0.90,1.0
10.0,-1247.77372070,0.96,1.0
12.0,-1287.09226074,0.98,1.0
15.0,-1295.35957520,1.00,1.0
17.0,-1308.13117188,1.00,1.0
20.0,-1309.02648438,1.00,1.0
22.0,-1307.66769775,1.00,1.0


In [21]:
pd.set_option('precision', 8)
results_df.groupby(['label', 'noisy_value']).mean()

loss  acc  top-k acc
label  noisy_value                               
16-QAM 2.0         -1111.75103149  0.8        1.0
       5.0         -1271.12095947  1.0        1.0
       7.0         -1350.15004883  1.0        1.0
       10.0        -1394.09505615  1.0        1.0
       12.0        -1468.94217529  1.0        1.0
       15.0        -1484.90805664  1.0        1.0
       17.0        -1489.05098877  1.0        1.0
       20.0        -1489.70736084  1.0        1.0
       22.0        -1489.53143311  1.0        1.0
       25.0        -1489.51304932  1.0        1.0
       27.0        -1487.57561035  1.0        1.0
       30.0        -1481.97727051  1.0        1.0
32-QAM 2.0         -1332.80288086  1.0        1.0
       5.0         -1262.84291382  0.9        1.0
       7.0         -1305.14174805  1.0        1.0
       10.0        -1356.92369385  1.0        1.0
       12.0        -1450.89708252  1.0        1.0
       15.0        -1456.30135498  1.0        1.0
       17.0        -1519.00357666  1.0        1.0
       20.0        -1521.97791748  1.0        1.0
       22.0        -1523.15098877  1.0        1.0
       25.0        -1521.42377930  1.0        1.0
       27.0        -1523.16452637  1.0        1.0
       30.0        -1515.49044189  1.0        1.0
4--PSK 2.0         -1144.06672363  1.0        1.0
       5.0         -1186.08837891  1.0        1.0
       7.0         -1207.01481934  1.0        1.0
       10.0        -1223.36691895  1.0        1.0
       12.0        -1229.96470947  1.0        1.0
       15.0        -1235.36196289  1.0        1.0
       17.0        -1231.71330566  1.0        1.0
       20.0        -1233.51334229  1.0        1.0
       22.0        -1228.92393799  1.0        1.0
       25.0        -1230.93673096  1.0        1.0
       27.0        -1225.53024902  1.0        1.0
       30.0        -1221.95905762  1.0        1.0
64-QAM 2.0         -1022.12596436  0.3        1.0
       5.0         -1051.64542847  0.2        1.0
       7.0         -1062.09993286  0.5        1.0
       10.0        -1080.63371582  0.8        1.0
       12.0        -1099.55115967  0.9        1.0
       15.0        -1108.52535400  1.0        1.0
       17.0        -1110.12156982  1.0        1.0
       20.0        -1110.50270996  1.0        1.0
       22.0        -1110.60142822  1.0        1.0
       25.0        -1111.32402344  1.0        1.0
       27.0        -1111.05988770  1.0        1.0
       30.0        -1110.75993652  1.0        1.0
8--QAM 2.0         -1038.83410645  0.9        1.0
       5.0         -1137.37695923  1.0        1.0
       7.0         -1175.02558594  1.0        1.0
       10.0        -1183.84921875  1.0        1.0
       12.0        -1186.10617676  1.0        1.0
       15.0        -1191.70114746  1.0        1.0
       17.0        -1190.76641846  1.0        1.0
       20.0        -1189.43109131  1.0        1.0
       22.0        -1186.13070068  1.0        1.0
       25.0        -1184.03568115  1.0        1.0
       27.0        -1183.63936768  1.0        1.0
       30.0        -1184.50264893  1.0        1.0